## Process

Whenever we use a computer, we are always interacting with programs like opening a browser, wrting on a word processor, or using command line interfaces. The operating system's abstraction for running a program is a process. These processes are both parent and child processes.

The Python subprocess module is used to launch child processes. Here, Python is the parent process and child could be anything from shell to the GUI applications.

Let's create an example. In this example we will create a simple Python program and we will use subprocess to call it. Although we don't need to call a Python program as a separate sub-process, it can as well be imported as a module but the main idea is to show how subprocess module be cross-platform in nature. 

The Python program will add the numbers with sleep time of 1 sec and print them. (A simple and a cool program)

In [3]:
import subprocess

In [14]:
subprocess.run(["python3", "adder_timer.py", "3"], check = True, timeout = 10)

CompletedProcess(args=['python3', 'adder_timer.py', '3'], returncode=0)

On executing run(), the program runs in the terminal (from where you have launched in the sub-process) and once it is done it returns an instance of CompletedProcess class with a returncode = 0. If there is some errors the resturncode is not equal to 0. Under the hood, run() makes a system call and it does not need a shell to do it.

With subprocess we can run any App like say, notepad. The following command opens up a notepad and when you close it, it again returns an instance of CompletedProcess class with a returncode = 0.

So, by default many modules (higher level of abstarction) uses subprocess() to do low-level operations.

* Exceptions Handling-
1. check = True raises the exception when returncode is not equals to 0. (CalledProcessError)
2. Some processes might take too long to run or may have hang indefinitely (TimeoutExpired). Giving a timeout parameter will end the sub-process. It is in seconds
3. FileNotFoundError if there is no file found, in our case say the adder_timer.py does not exist.

In [9]:
subprocess.run(['notepad'])

CompletedProcess(args=['notepad'], returncode=0)

#### Exception Handling

In [17]:
try:
    subprocess.run(
        ["python3", "adder_timer.py", "5"], timeout = 10
    )
except FileNotFoundError as exc:
    print(f"Process failed to locate the executable file.\n{exc}")
except subprocess.CalledProcessError as exc:
    print(f"Process failed to return a success returncode"
         f"The process returned {exc.returncode}\n{exc}"
         )
except subprocess.TimeoutExpired as exc:
    print(f"Process timed out.\n{exc}")

### Pipes and Shells

When the processes get initialized, they use the following three special streams -

1. Reads stdin for input
2. Writes to stdout
3. Writes to stderr for reporting the errors.

Now if we want to feed stdout from one process to the stdin of another process, pipe or pipeline is a special stream that does that. It has two file handles one is read-only and another one is write-only. In this way it joins or creates a pipe to connect a byte stream from one process to another. 

You might be aware of pipes or pipelines in UNIX based operating system. Also there is an equivalent in Windows using powershell. These pipe operations can be also done using subporcess module

In [20]:
#ls /usr/bin | grep python

In [22]:
#!ls "C:\Program Files" | Out-String -stream | Select-String windows

In [27]:
add_number_process = subprocess.run(
                        ["python3", "adder_timer.py", "3"], capture_output = True)
add_number_process.stdout

b'Starting timer of 3 seconds\r\n0 1 3 \r\nDone!!\r\n'

Above we have used capture_output = True to able to access stdout which is a byte stream. This is equivalent to setting the stdout and stderr parameters to using a pipe from subprocess

In [32]:
add_number_process = subprocess.run(
                        ["python3", "adder_timer.py"],
                        stdout = subprocess.PIPE,
                        stderr = subprocess.PIPE)

In [33]:
add_number_process.stdout

b''

In [34]:
add_number_process.stderr

b'usage: adder_timer.py [-h] time\r\nadder_timer.py: error: the following arguments are required: time\r\n'

In [35]:
add_number_process = subprocess.run(
                        ["python3", "adder_timer.py", "3"],
                        stdout = subprocess.PIPE,
                        stderr = subprocess.PIPE)

In [38]:
add_number_process.stdout

b'Starting timer of 3 seconds\r\n0 1 3 \r\nDone!!\r\n'

In [39]:
add_number_process.stderr

b''

If we are planning to hold large quantities of data in the pipe (as a buffer) it won't be able to hold it as pipe buffers have limited capacity. The option is to pass a file object to the stdout stream parameters.

In [42]:
from tempfile import TemporaryFile

# Returns an object with a file-like interface; the name of the file
# is accessible as its 'name' attribute.  The file will be automatically
# deleted when it is closed unless the 'delete' argument is set to False.

In [45]:
with TemporaryFile() as f:
    my_process = subprocess.run(["python3", "adder_timer.py", ], 
                                stdout = f,
                                stderr = subprocess.PIPE)
    f.seek(0) # start of the stream position
    print(f.read().decode("utf-8"))

In [48]:
my_process.stderr

b'usage: adder_timer.py [-h] time\r\nadder_timer.py: error: the following arguments are required: time\r\n'

In [50]:
with TemporaryFile() as f:
    my_process = subprocess.run(["python3", "adder_timer.py", "3"], 
                                stdout = f,
                                stderr = subprocess.PIPE)
    f.seek(0)
    print(f.read().decode("utf-8"))

Starting timer of 3 seconds
0 1 3 
Done!!



In [51]:
my_process.stderr

b''

### Popen Class

Will go over it.